In [39]:
import pandas as pd
import numpy as np
from keras.models import load_model

In [40]:
df = pd.read_csv("data/nfl.csv")

# Grab dummy variables
dummy_vars = pd.get_dummies(df[["opp", "team"]])

# Add our dummy columns into the features df
df[list(dummy_vars.columns)] = dummy_vars

# Define the features
feature_df = df.drop(["result", "date", "opp", "team", "margin",
                          "points", "points_allowed", "total_points"], axis=1)

# Grab column names
columns = list(feature_df.columns)

zeros = np.zeros(shape=(1, 82))

default_df = pd.DataFrame(zeros, columns=columns)

oriented = default_df.to_dict(orient="records")

In [41]:
default_df

,third_per,third_per_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
model_input_df = default_df.copy()

In [43]:
model_input_dict = {'team': 'ARI', 'opp': 'ARI', 'third_per': '5753', 'third_per_allowed': '7534', 'TOP': '453', 'first_downs': '7532', 'first_downs_allowed': '75345', 'ha': 'Home', 'pass_yards': '53786', 'pass_yards_allowed': '53786', 'penalty_yards': '453453', 'plays': '786432', 'rush_yards': '753453', 'rush_yards_allowed': '43788', 'sacked': '453', 'sacks': '789', 'takeaways': '721', 'total_yards': '7893', 'total_yards_allowed': '45645', 'turnovers': '76'}

In [44]:
columns = list(model_input_df.columns)

In [45]:
for column in columns:
    for key, value in model_input_dict.items():
        if key == column:
            model_input_df[column] = value
        elif f"{key}_{value}" == column:
            model_input_df[column] = 1

In [46]:
model_input_df

,third_per,third_per_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,5753,7534,453,7532,75345,Home,53786,53786,453453,786432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
ha_value = model_input_df["ha"][0]
if ha_value == "Home":
    model_input_df["ha"] = 0
elif ha_value == "Away":
    model_input_df["ha"] = 1

In [48]:
model_input_df

,third_per,third_per_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,5753,7534,453,7532,75345,0,53786,53786,453453,786432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Load model

In [49]:
deep_model = load_model("models/deep_neural_model_trained.h5")

# Predict result with user input values

In [50]:
encoded_prediction = deep_model.predict_classes(model_input_df)

In [51]:
encoded_prediction

array([0], dtype=int64)

In [52]:
print(f"Predicted class: {encoded_prediction}")

Predicted class: [0]


# Make sure model is correct

In [73]:
feature_df.head()

,third_per,third_per_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.350000,18,19,away,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0
1,0.285714,0.307692,36.183333,19,18,home,321,234,60,70,...,0,0,0,0,1,0,0,0,0,0
2,0.363636,0.400000,29.116667,16,19,away,163,213,27,56,...,0,0,0,0,0,0,0,0,0,0
3,0.400000,0.363636,30.883333,19,16,home,213,163,35,65,...,0,0,0,0,0,0,0,0,0,0
4,0.250000,0.333333,26.550000,10,16,away,227,221,39,51,...,0,0,0,0,0,0,0,0,0,0


In [74]:
# Dummify ha where away is 0 and home is 1
feature_df["ha"] = feature_df["ha"].map(lambda x: 0 if x == "home" else 1)

In [84]:
test_check_input = feature_df.iloc[[0,1,2,3,4,5,6,7,8,9]]

In [85]:
test_check_input

,third_per,third_per_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.350000,18,19,1,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0
1,0.285714,0.307692,36.183333,19,18,0,321,234,60,70,...,0,0,0,0,1,0,0,0,0,0
2,0.363636,0.400000,29.116667,16,19,1,163,213,27,56,...,0,0,0,0,0,0,0,0,0,0
3,0.400000,0.363636,30.883333,19,16,0,213,163,35,65,...,0,0,0,0,0,0,0,0,0,0
4,0.250000,0.333333,26.550000,10,16,1,227,221,39,51,...,0,0,0,0,0,0,0,0,0,0
5,0.333333,0.250000,33.450000,16,10,0,221,227,27,63,...,0,0,0,0,0,0,0,0,0,0
6,0.428571,0.250000,33.366667,19,17,1,85,179,38,63,...,0,0,0,0,0,0,0,0,0,0
7,0.250000,0.428571,26.633333,17,19,0,179,85,66,60,...,0,0,0,0,0,0,0,0,0,0
8,0.400000,0.571429,26.066667,12,21,1,114,294,24,55,...,0,0,0,0,0,0,0,0,0,0
9,0.571429,0.400000,33.933333,21,12,0,294,114,17,70,...,0,0,0,0,0,0,0,0,0,0


In [87]:
deep_model.predict_classes(test_check_input)

array([0, 2, 0, 2, 2, 2, 0, 0, 0, 2], dtype=int64)

In [89]:
df["result"][:10]

0    Loss
1     Win
2    Loss
3     Win
4     Win
5    Loss
6     Win
7    Loss
8    Loss
9     Win
Name: result, dtype: object

In [86]:
print(f"Predicted class: {deep_model.predict_classes(test_check_input)}")

Predicted class: [0 2 0 2 2 2 0 0 0 2]


#### 0 means loss and 2 means win

# Build form

In [53]:
features = ["Team", "Opponent", "Third", "Third Allowed", "TOP", "First Downs",
               "First Downs Allowed", "Home or Away", "Pass Yards", "Pass Yards Allowed",
               "Penalty Yards", "Plays", "Rush Yards", "Rush Yards Allowed", "Sacked", "Sacks",
               "Takeaways", "Total Yards", "Total Yards Allowed", "Turnovers"]

form_names = ["team", "opponent", "third", "third_allowed", "top", "first_downs",
                  "first_downs_allowed", "ha", "pass_yards", "pass_yards_allowed", "penalty_yards",
                  "plays", "rush_yards", "rush_yards_allowed", "sacked", "sacks", "takeaways",
                  "total_yards", "total_yards_allowed", "turnovers"]

placeholders = ["team", "opponent", "third", "third-allowed", "top", "first-downs",
                    "first-downs-allowed", "ha", "pass-yards", "pass-yards-allowed", "penalty-yards",
                    "plays", "rush-yards", "rush-yards-allowed", "sacked", "sacks", "takeaways",
                    "total-yards", "total-yards-allowed", "turnovers"]

In [54]:
form_df = pd.DataFrame({
    "feature": features,
    "form_name": form_names,
    "placeholder": placeholders
})

In [55]:
form_df.head()

,feature,form_name,placeholder
0,Team,team,team
1,Opponent,opponent,opponent
2,Third,third,third
3,Third Allowed,third_allowed,third-allowed
4,TOP,top,top


In [56]:
form_data = form_df.to_dict(orient="records")

In [57]:
form_data

[{'feature': 'Team', 'form_name': 'team', 'placeholder': 'team'},
 {'feature': 'Opponent', 'form_name': 'opponent', 'placeholder': 'opponent'},
 {'feature': 'Third', 'form_name': 'third', 'placeholder': 'third'},
 {'feature': 'Third Allowed',
  'form_name': 'third_allowed',
  'placeholder': 'third-allowed'},
 {'feature': 'TOP', 'form_name': 'top', 'placeholder': 'top'},
 {'feature': 'First Downs',
  'form_name': 'first_downs',
  'placeholder': 'first-downs'},
 {'feature': 'First Downs Allowed',
  'form_name': 'first_downs_allowed',
  'placeholder': 'first-downs-allowed'},
 {'feature': 'Home or Away', 'form_name': 'ha', 'placeholder': 'ha'},
 {'feature': 'Pass Yards',
  'form_name': 'pass_yards',
  'placeholder': 'pass-yards'},
 {'feature': 'Pass Yards Allowed',
  'form_name': 'pass_yards_allowed',
  'placeholder': 'pass-yards-allowed'},
 {'feature': 'Penalty Yards',
  'form_name': 'penalty_yards',
  'placeholder': 'penalty-yards'},
 {'feature': 'Plays', 'form_name': 'plays', 'placehold